# MCTS

In [210]:
def search(obs, target_player=None):    
    current_player = env.current_player
    if True or target_player is None or target_player == env.current_player:        
        if target_player is None:
            target_player = env.current_player
            
        state = env.get_state()
        n = len(env.players[env.current_player].hand_cards)   
        p = 0
        distr = np.zeros((n,))

        for i in range(n):
            card_id = env.players[env.current_player].hand_cards[i].id
            obs, rew, is_done, _ = env.step(card_id)

            if is_done:
                distr[i] = (rew[0] > 0)
            else:
                distr[i] = ((1 - search(obs, target_player)[0]) if current_player != env.current_player else search(obs, target_player)[0])
            env.set_state(state)
            p += distr[i]
        
        return distr.mean() if target_player == env.current_player else distr.mean(), distr
    else:
        state = env.get_state()
        p = calc_correct_output(env.players[env.current_player].hand_cards, env.table_card, env.players[env.current_player].tricks, env.players[1 - env.current_player].tricks)
        env.set_state(state)
        step = env.players[env.current_player].hand_cards[np.argmax(p)].id
        
        obs, rew, is_done, _ = env.step(step)
        if is_done:
            return (rew[0] > 0), []
        else:
            return ((1 - search(obs, target_player)[0]) if current_player != env.current_player else search(obs, target_player)[0]), []

In [227]:
class State:
    def __init__(self, p, env_state, current_player, end_v=0, is_root=True):
        self.childs = []
        self.n = 0
        self.w = 0
        self.v = 0
        self.p = p
        self.env_state = env_state
        self.end_v = end_v
        self.current_player = current_player
        self.is_root = is_root
        
    def is_leaf_node(self):
        return len(self.childs) == 0
    
    def size(self):
        size = 1
        for child in self.childs:
            size += child.size()
        return size

def calc_q(state, player):
    if state.end_v != 0:
        return state.end_v * (-1 if player == 1 else 1), 1
    elif state.current_player == player:
        if state.n is 0:
            return 0, 0
        else:
            return state.w / state.n, state.n
    else:
        q_sum = 0
        n_sum = 0
        for child in state.childs:            
            q_new, n = calc_q(child, player)
            q_sum += q_new * n
            n_sum += n
        return 0 if n_sum is 0 else q_sum / n_sum, n_sum

def mcts_sample(state, use_model=True):
    
    if state.is_leaf_node():
        if state.end_v != 0:
            v = state.end_v
            player = -1
        else:
            env.set_state(state.env_state)

            if use_model:
                p, v = model.predict_single(env.regenerate_obs())
            else:
                p, v = [1] *32, [0]
                
            hand_cards = env.players[env.current_player].hand_cards[:]        
            current_player = env.current_player
            for card in hand_cards:
                obs, rew, is_done, _ = env.step(card.id)
                
                new_state = State(p[card.id], env.get_state(), env.current_player, 0 if not is_done else (1 if rew[0] > 0 else -1) * (-1 if current_player is 1 else 1), False)
                state.childs.append(new_state)

                env.set_state(state.env_state)

            v = v[0]  
            state.v = v
            player = state.current_player
    else:
        n_sum = 0
        for child in state.childs:
            n_sum += child.n
            
        if state.is_root:
            epsilon = EPSILON
            nu = np.random.dirichlet([ALPHA] * len(state.childs))
          #  print("")
        else:
            epsilon = 0
            nu = [0] * len(state.childs)

        max_u = 0
        max_child = None
        if not OBJECTIVE_OPPONENT or state.current_player == 0:
            i = 0        
            for child in state.childs:
                u = calc_q(child, state.current_player)[0]
                u += EXPLORATION * ((1 - epsilon) * child.p + epsilon * nu[i]) * np.sqrt(n_sum) / (1 + child.n)

                #f state.is_root:
                #   print(i, u)            

                if max_child is None or u > max_u:
                    max_u, max_child = u, child
                i += 1
        else:
            p = [child.p for child in state.childs]
            p = np.exp(p - np.max(p))
            p /= p.sum(axis=0)
            max_child = state.childs[np.random.choice(np.arange(0, len(p)), p=p)]
            
        v, player = mcts_sample(max_child, use_model)
     
    if player == state.current_player:        
        state.w += v 
        state.n += 1
    elif player == -1:       
        state.w += v * (-1 if state.current_player == 1 else 1)
        state.n += 1        
    return v, player


In [240]:

def mcts_game_step(root, steps=MCTS_SIMS, use_model=True):
    for i in range(steps):
        mcts_sample(root, use_model)
        
    if not OBJECTIVE_OPPONENT or root.current_player == 0:
        p = []
        for child in root.childs:
            p.append(calc_q(child, root.current_player)[0])
    else:
        p = [child.p for child in root.childs]
    p = np.exp(p - np.max(p))
    p /= p.sum(axis=0)
    return np.random.choice(np.arange(0, len(p)), p=p), p

def generate_key():
    card_ids = []
    for card in env.players[env.current_player].hand_cards:
        card_ids.append(card.id)
    card_ids.sort()
    return str(card_ids) + "-" + (str(env.table_card.id) if env.table_card is not None else "") + "-" + str(env.players[env.current_player].tricks) + "-" + str(env.players[1 - env.current_player].tricks)
    
def clean_unfinished_samples(model):
    global sample_inputs, sample_outputs, next_index, number_of_samples, finished_sample_numbers
    number = 0
    for hand_cards, unfinished_samples_per_card_number in enumerate(unfinished_samples):
        keys_to_remove = []
        for key, sample in unfinished_samples_per_card_number.items():
            if len(sample[1]) >= SAMPLES_PER_STATE:
                if sample[0][2][1] == 0:
                    sample_inputs[0][next_index] = sample[0][0]
                    sample_inputs[1][next_index] = sample[0][1]

                    sample_outputs[0][next_index].fill(0)
                    sample_outputs[1][next_index].fill(0)

                    for single_sample in sample[1]:
                        sample_outputs[0][next_index] += single_sample[1]
                        sample_outputs[1][next_index] += single_sample[0]

                    sample_outputs[0][next_index] = np.divide(sample_outputs[0][next_index], len(sample[1]))
                    sample_outputs[1][next_index] = np.divide(sample_outputs[1][next_index], len(sample[1]))

                    if hasattr(model, 'memorize'):
                        model.memorize([sample_inputs[0][next_index], sample_inputs[1][next_index]], [sample_outputs[0][next_index], sample_outputs[1][next_index]])

                    finished_sample_numbers[hand_cards] += 1
                    next_index += 1
                    number += 1
                    next_index %= MEMORY_SIZE
                    number_of_samples = max(number_of_samples, next_index)
                keys_to_remove.append(key)
        
        for key in keys_to_remove:
            del unfinished_samples_per_card_number[key]
    return number

def mcts_game():
    global unfinished_samples, finished_sample_numbers
    
    next_hand_card_size = finished_sample_numbers.index(min(finished_sample_numbers))
            
    if len(unfinished_samples[next_hand_card_size]) == 0:
        #n = random.choice(eval_cache_initial_states)
        #game = generate_game_from_obs(eval_cache_input[n])
        #set_env(*game)
        obs = env.reset()    
        env.current_player = random.randint(0, 1)
    else:
        val = iter(unfinished_samples[next_hand_card_size].values())
        #next(val)
        env.set_state(next(val)[0][-1])
        
        opponent_cards = env.players[1 - env.current_player].hand_cards
        number_of_opponent_cards = len(opponent_cards)
        env.cards_left.extend(opponent_cards)
        random.shuffle(env.cards_left)
        
        opponent_cards.clear()
        for hand_card in range(number_of_opponent_cards):
            opponent_cards.append(env.cards_left.pop())
        
        obs = env.regenerate_obs()
        #print(generate_key())
   # env.render('human')
    #print(obs)
        
    root = State(1, env.get_state(), env.current_player)
    is_done = False
    values = []
    
    while not is_done:
        key = generate_key()
                
        if not key in unfinished_samples[5 - len(env.players[env.current_player].hand_cards)]:
            unfinished_samples[5 - len(env.players[env.current_player].hand_cards)][key] = [[np.array(obs[0]), np.array(obs[1]), env.get_state()], []]
        current_sample = unfinished_samples[5 - len(env.players[env.current_player].hand_cards)][key]
        
        current_sample[1].append([1 if env.current_player is 0 else -1])    
        values.append(current_sample[1][-1])

        game_state = env.get_state()
        a, p = mcts_game_step(root)
        env.set_state(game_state)
        
        output = np.zeros((32,))
        for i, card in enumerate(env.players[env.current_player].hand_cards):
            output[card.id] = p[i]    
        current_sample[1][-1].append(output)
                
        last_player = env.current_player
        obs, rew, is_done, _ = env.step(env.players[env.current_player].hand_cards[a].id)
        root = root.childs[a]
        root.is_root = True
                
    for val in values:
        val[0] *= (1 if last_player is 0 else -1) * (1 if rew[0] > 0 else -1)
        
    return clean_unfinished_samples(model)
        
def mcts_generate():
    
    number = 0
    with tqdm(total=EPISODES * 5) as progress_bar:
        while number < EPISODES * 5:
            number += mcts_game()
            progress_bar.update(number - progress_bar.n)
        
    #postprocess_samples()

def reset_samples():
    global sample_outputs, sample_inputs, next_index, number_of_samples, unfinished_samples, finished_sample_numbers
    sample_outputs = [np.zeros((MEMORY_SIZE, 32), dtype=float), np.zeros((MEMORY_SIZE, 1), dtype=float)]
    sample_inputs = [np.zeros((MEMORY_SIZE, 4, 8, 2), dtype=int), np.zeros((MEMORY_SIZE, 4), dtype=int)]
    unfinished_samples = [{}, {}, {}, {}, {}]
    finished_sample_numbers = [0, 0, 0, 0, 0]
    next_index = 0
    number_of_samples = 0
    
def draw_tree(root, tree_depth=5, tree_path=[]):
    dot = pydot.Dot()
    dot.set('rankdir', 'TB')
    dot.set('concentrate', True)
    dot.set_node_defaults(shape='record')

    create_nodes(root, dot, tree_depth, tree_path)

   # print("Root: " + str(root.end_v if root.n is 0 else root.w / root.n) + " / " + str(root.n))
   # for child in root.childs:
   #     print( str(child.end_v if child.n is 0 else child.w / child.n) + " / " + str(child.n) + " p: " + str(child.p))

    # render pydot by calling dot, no file saved to disk
    png_str = dot.create_png(prog='dot')
    dot.write_svg('tree.svg')

    # treat the dot output string as an image file
    sio = BytesIO()
    sio.write(png_str)
    sio.seek(0)
        
    # plot the image
    fig, ax = plt.subplots(figsize=(18, 5))
    ax.imshow(plt.imread(sio), interpolation="bilinear")
    
def create_nodes(root, dot, tree_depth, tree_path, id=0):    
    text = "N: " + str(root.n) + " (" + str(root.current_player) + ')\n'
    text += "Q: " + str(root.end_v if root.end_v is not 0 or root.n is 0 else root.w / root.n) + '\n'
    text += "P: " + str(root.p) + '\n'
    text += "V: " + str(root.v)
    
    node = pydot.Node(str(id), label=text)
    dot.add_node(node)    
    id += 1

    if tree_depth > 1:
        i = 0
        for child in root.childs:
            if len(tree_path) == 0 or (type(tree_path[0]) == list and i in tree_path[0] or type(tree_path[0]) == int and tree_path[0] == i):
                child_node, id = create_nodes(child, dot, tree_depth - 1, tree_path if len(tree_path) == 0 else tree_path[1:], id)
                dot.add_edge(pydot.Edge(node.get_name(), child_node.get_name()))
            i+=1
        
    return node, id
    
        
def draw_path(root):
    state = root
    while True:     
        print(str(state.current_player) + ": " + str(state.end_v if state.n is 0 else state.w / state.n) + " / " + str(state.n))
        
        if state.is_leaf_node():
            break
    
        max_u = 0
        max_child = None
        for child in state.childs:
            if child.end_v != 0:
                u = child.end_v
            elif child.n > 0:
                u = child.w / child.n 
            else:
                u = -1
                
            if max_child is None or u > max_u:
                max_u, max_child = u, child
        
        state = max_child


In [328]:
reset_samples()

In [331]:
mcts_game()

[0.37373737 0.37373737 0.25252525]
[0.52205882 0.47794118]
[0.42352941 0.57647059]


1

In [238]:
next_index

0

In [315]:
unfinished_samples[2]['[4, 9, 25]-19-1-1'][1]

[[-1, array([0.        , 0.        , 0.        , 0.        , 0.32323232,
         0.        , 0.        , 0.        , 0.        , 0.36363636,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.31313131, 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        ])],
 [-1, array([0.        , 0.        , 0.        , 0.        , 0.34343434,
         0.        , 0.        , 0.        , 0.        , 0.33333333,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.32323232, 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        ])],
 [1, array([0.        , 0.        , 0.        , 0.        , 0.94949495,
         0.        ,

In [332]:
sample_outputs[0][0]

array([0.        , 0.        , 0.        , 0.        , 0.34680135,
       0.        , 0.        , 0.        , 0.        , 0.38383838,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.26936027, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        ])